**[MST-01]** 모듈을 임포트하고 난수의 시드를 설정한다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

**[MST-02]** MNIST 데이터 세트를 준비한다.

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MST-03]** 단층 신경망을 표현하는 클래스를 정의한다.

In [18]:
class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()

    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
        
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]),
                             name='weights')        
            b1 = tf.Variable(tf.zeros([num_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name='hidden1')
        
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name='weights')
            b0 = tf.Variable(tf.zeros([10]), name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')

        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)

        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                              tf.float32), name='accuracy')

        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("weights_hidden", w1)
        tf.summary.histogram("biases_hidden", b1)
        tf.summary.histogram("weights_output", w0)
        tf.summary.histogram("biases_output", b0)
                
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
            
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_sl_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

**[MST-04]** 텐서보드용 데이터 출력 디렉토리를 삭제해서 초기화해둔다.

In [4]:
!rm -rf /tmp/mnist_sl_logs

**[MST-05]** 파라미터 최적화를 2000회 반복한다. 테스트 세트에 대해 약 97%의 정답률을 얻을 수 있다.

In [19]:
nn = SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x: batch_xs, nn.t: batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run(
            [nn.summary, nn.loss, nn.accuracy],
            feed_dict={nn.x:mnist.test.images, nn.t: mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)

Step: 100, Loss: 2718.882568, Accuracy: 0.919100
Step: 200, Loss: 2226.209229, Accuracy: 0.931200
Step: 300, Loss: 1815.973755, Accuracy: 0.943800
Step: 400, Loss: 1624.048584, Accuracy: 0.948000
Step: 500, Loss: 1528.272461, Accuracy: 0.954900
Step: 600, Loss: 1415.506470, Accuracy: 0.957800
Step: 700, Loss: 1345.908569, Accuracy: 0.958800
Step: 800, Loss: 1269.461548, Accuracy: 0.962000
Step: 900, Loss: 1289.555420, Accuracy: 0.962400
Step: 1000, Loss: 1298.363403, Accuracy: 0.961100
Step: 1100, Loss: 1257.837891, Accuracy: 0.962100
Step: 1200, Loss: 1103.579956, Accuracy: 0.966300
Step: 1300, Loss: 1131.564331, Accuracy: 0.964600
Step: 1400, Loss: 1062.740601, Accuracy: 0.967100
Step: 1500, Loss: 1164.019409, Accuracy: 0.965200
Step: 1600, Loss: 1030.684570, Accuracy: 0.969400
Step: 1700, Loss: 957.296387, Accuracy: 0.972800
Step: 1800, Loss: 977.647339, Accuracy: 0.970100
Step: 1900, Loss: 1104.120483, Accuracy: 0.968200
Step: 2000, Loss: 1012.303101, Accuracy: 0.970900
